In [ ]:
import torch
import torch.nn as nn                # für nn layers
import torch.nn.functional as F       
import torch.optim as optim            
from torch.utils.data import Dataset, DataLoader 

#
import torchaudio
import torchaudio.transforms as T   
import numpy as np                     
import pandas as pd                   

import matplotlib.pyplot as plt

In [5]:
import pandas as pd 

data = pd.read_csv("ML2_Birds/processed_data.csv")

In [6]:
data.head()

,primary_label,secondary_labels,type,filename,rating,latitude,longitude
0,0,[''],[''],1139490/CSA36385.ogg,0.0,7.3206,-73.7128
1,0,[''],[''],1139490/CSA36389.ogg,0.0,7.3206,-73.7128
2,1,[''],[''],1192948/CSA36358.ogg,0.0,7.3791,-73.7313
3,1,[''],[''],1192948/CSA36366.ogg,0.0,7.2800,-73.8582
4,1,[''],[''],1192948/CSA36373.ogg,0.0,7.3791,-73.7313


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28564 entries, 0 to 28563
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   primary_label     28564 non-null  int64  
 1   secondary_labels  28564 non-null  object 
 2   type              28564 non-null  object 
 3   filename          28564 non-null  object 
 4   rating            28564 non-null  float64
 5   latitude          27755 non-null  float64
 6   longitude         27755 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader #instanzen verarbeitung 
import pandas as pd # df frame erstellung 
import os
import ast # zum parsen
from PIL import Image 
from torchvision import transforms # Für Bild-Transformationen

metadata = data
spectogramm = "Data_BIRDCLEF/birdclef-2025/train_audio"

class SpectogramDataset(Dataset):
    def __init__(self, metadata_df:pd.DataFrame, audio_dir:str, label_to_idx:dict, transform):
        self.metadata_df = metadata_df
        self.audio_dir = audio_dir
        self.label_to_idx = label_to_idx
        self.num_classes = len(label_to_idx)
        self.transform = transform
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((128,128)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])
        else: 
            self.transform = transform


    def __len__(self):
        return len(self.metadata_df)
    
    def __getitem__(self, idx:int):
        row = self.metadata_df.iloc[idx]
        ogg_original = row["filename"]
        primary_label = str(row["primary_label"])
        secondary_labels = str(row["secondary_labels"])

        spectogramm, _ = os.path.splitext(ogg_original) #os.path.splitext trennt den Pfad in (root, ext)
        spectogramm_file_jpg = spectogramm + ".jpg" #change to jpeg if it isnt jpg- darf ich nicht vergessen!!!!!!!!!!!

        spectogramm_path = os.path.join(self.audio_dir, spectogramm_file_jpg)

        try: 
            spectogram_image = Image.open(spectogramm_path).convert("L") # Bild in Graustufenmodus L laden, da Spektogramme keine Farbkanäle haben sollten
        except FileNotFoundError: #Wirft einen Fehler wenn die Datei nicht gefunden wurde
            print(f"Spektogrammdatei nicht gefunden: {spectogramm_path} (Original OGG: {ogg_original}). Rückgabe von DummySample")
            
            dummy_array = np.zeros((128,128), dtype=np.uint8) #Werte von 0 (Schwarzes Bild) in shape von den anderne Spektogrammen
            dummy_pil = Image.fromarray(dummy_array, mode="L")
            dummy_labels = torch.zeros(self.num_classes, dtype=torch.float32)
            dummy_image_tensor = self.transform(dummy_pil)
        
            return dummy_image_tensor, dummy_labels


        except Exception as e: #Fängt alle anderen Fehler auf die aufkommen 
            print(f"Warnung: Fehler beim Laden/Verarbeiten vom Bild: {spectogramm_path}, Fehler: {e}")
            dummy_array = np.zeros((128,128), dtype=np.uint8) #Werte von 0 (Schwarzes Bild)
            dummy_pil = Image.fromarray(dummy_array, mode="L")
            dummy_labels = torch.zeros(self.num_classes, dtype=torch.float32)
            dummy_image_tensor = self.transform(dummy_pil)
            return dummy_image_tensor, dummy_labels
        

        label_tensor = torch.zeros(self.num_classes, dtype=torch.float32)
        #Setzen des Primary labels 
        if primary_label in self.label_to_idx:
            label_tensor[self.label_to_idx[primary_label]] = 1.0

        #Secondary label muss geparsed d.h aus "[123, 123]" wird [123, 123] werden, deswegen ast
        try:
            secondary_l_list = ast.literal_eval(secondary_labels)
            if isinstance(secondary_l_list, list): #check up ob ast wirklich eine liste ausgegeben hat 
                for sec in secondary_l_list:
                    if sec in self.label_to_idx:
                        label_tensor[self.label_to_idx[sec]] = 1.0

        except:
            pass

        
        spectogramm_tensor = self.transform(spectogram_image)
        return spectogramm_tensor, label_tensor



        



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader #instanzen verarbeitung 
import pandas as pd # df frame erstellung 
import os
import ast # zum parsen
from PIL import Image 
from torchvision import transforms # Für Bild-Transformationen

metadata = data
spectogramm = "Data_BIRDCLEF/birdclef-2025/train_audio"

class SpectogramDataset(Dataset):
    def __init__(self, metadata_df:pd.DataFrame, audio_dir:str, label_to_idx:dict, transform):
        self.metadata_df = metadata_df
        self.audio_dir = audio_dir
        self.label_to_idx = label_to_idx
        self.num_classes = len(label_to_idx)
        self.transform = transform
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((128,128)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])
        else: 
            self.transform = transform


    def __len__(self):
        return len(self.metadata_df)
    
    def __getitem__(self, idx:int):
        row = self.metadata_df.iloc[idx]
        ogg_original = row["filename"]
        primary_label = str(row["primary_label"])
        secondary_labels = str(row["secondary_labels"])

        spectogramm, _ = os.path.splitext(ogg_original) #os.path.splitext trennt den Pfad in (root, ext)
        spectogramm_file_jpg = spectogramm + ".jpg" #change to jpeg if it isnt jpg- darf ich nicht vergessen!!!!!!!!!!!

        spectogramm_path = os.path.join(self.audio_dir, spectogramm_file_jpg)

        try: 
            spectogram_image = Image.open(spectogramm_path).convert("L") # Bild in Graustufenmodus L laden, da Spektogramme keine Farbkanäle haben sollten
        except FileNotFoundError: #Wirft einen Fehler wenn die Datei nicht gefunden wurde
            print(f"Spektogrammdatei nicht gefunden: {spectogramm_path} (Original OGG: {ogg_original}). Rückgabe von DummySample")
            
            dummy_array = np.zeros((128,128), dtype=np.uint8) #Werte von 0 (Schwarzes Bild) in shape von den anderne Spektogrammen
            dummy_pil = Image.fromarray(dummy_array, mode="L")
            dummy_labels = torch.zeros(self.num_classes, dtype=torch.float32)
            dummy_image_tensor = self.transform(dummy_pil)
        
            return dummy_image_tensor, dummy_labels


        except Exception as e: #Fängt alle anderen Fehler auf die aufkommen 
            print(f"Warnung: Fehler beim Laden/Verarbeiten vom Bild: {spectogramm_path}, Fehler: {e}")
            dummy_array = np.zeros((128,128), dtype=np.uint8) #Werte von 0 (Schwarzes Bild)
            dummy_pil = Image.fromarray(dummy_array, mode="L")
            dummy_labels = torch.zeros(self.num_classes, dtype=torch.float32)
            dummy_image_tensor = self.transform(dummy_pil)
            return dummy_image_tensor, dummy_labels
        

        label_tensor = torch.zeros(self.num_classes, dtype=torch.float32)
        #Setzen des Primary labels 
        if primary_label in self.label_to_idx:
            label_tensor[self.label_to_idx[primary_label]] = 1.0

        #Secondary label muss geparsed d.h aus "[123, 123]" wird [123, 123] werden, deswegen ast
        try:
            secondary_l_list = ast.literal_eval(secondary_labels)
            if isinstance(secondary_l_list, list): #check up ob ast wirklich eine liste ausgegeben hat 
                for sec in secondary_l_list:
                    if sec in self.label_to_idx:
                        label_tensor[self.label_to_idx[sec]] = 1.0

        except:
            pass

        
        spectogramm_tensor = self.transform(spectogram_image)
        return spectogramm_tensor, label_tensor



        



In [ ]:
metadata = pd.read_csv("ML2_Birds/processed_data.csv")
unique_labels = sorted(metadata["primary_label"].astype(str).unique())
label_to_index = {label: idx for idx,label in enumerate(unique_labels)}
index_to_label = {idx: label for idx,label in enumerate(unique_labels)}
print(list(unique_labels)[:10])
#kann auch numerisch sorten theoretisch

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106']


In [ ]:
import pandas as pd

taxonomy_df = pd.read_csv("Data_BIRDCLEF/birdclef-2025/taxonomy.csv")

id_to_name = {}
for index, row in taxonomy_df.iterrows():
    primary_label = row["primary_label"]
    common_name = row["common_name"] 
    id_to_name[primary_label] = common_name

#für die visualisierung später 

print(list(id_to_name.items())[:5])

[('1139490', 'Ragoniella pulchella'), ('1192948', 'Oxyprora surinamensis'), ('1194042', 'Copiphora colombiae'), ('126247', 'Spotted Foam-nest Frog'), ('1346504', 'Neoconocephalus brachypterus')]


In [32]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    metadata,
    test_size=0.2,
    stratify=metadata["primary_label"], #this ensures that we keep the same proportions as we have in the dataset if 900 are a and 100 b we will have the same split in the test and train sets 
    random_state=42
)
train_df.head()
test_df.head()

,primary_label,secondary_labels,type,filename,rating,latitude,longitude
14604,134,"['strcuc1', 'socfly1', 'whtdov', 'yeofly1']",['song'],pirfly1/XC333930.ogg,3.5,15.5927,-92.8492
25900,190,[''],[''],whtdov/iNat819333.ogg,0.0,8.5808,-83.3835
16701,149,[''],['call'],rufmot1/XC250538.ogg,4.0,-1.0350,-77.3990
5509,84,[''],['call'],bubwre1/XC704163.ogg,3.0,-21.7557,-48.8326
7468,97,[''],['song'],compau/XC310847.ogg,5.0,5.6352,-75.8609


In [ ]:
metadata = pd.read_csv("ML2_Birds/processed_data.csv")
audio_dir="Data_BIRDCLEF/birdclef-2025/train_audio"


train_dataset = SpectogramDataset(
    metadata_df=train_df,
    audio_dir="Data_BIRDCLEF/birdclef-2025/train_audio",
    label_to_idx=label_to_index,
    transform=None
)

training_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
#CHATGPT Version weil meine nicht mehr funktioniert 

class TensorChunkDataset(Dataset):
    def __init__(self, tensor_dir, label_to_idx, transform=None):
        self.tensor_dir = tensor_dir
        self.label_to_idx = label_to_idx
        self.num_classes = len(label_to_idx)
        self.transform = transform
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((128,128)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])
        else: 
            self.transform = transform

        # Build index of (label_str, chunk_idx)
        self.samples = []
        for file in os.listdir(tensor_dir):
            if file.endswith(".pt"):
                label = file.replace(".pt", "")
                path = os.path.join(tensor_dir, file)
                data = torch.load(path)  # Shape: (N, 1, 128, 128)
                for i in range(data.shape[0]):
                    self.samples.append((label, i))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        label, chunk_idx = self.samples[idx]
        chunk_path = os.path.join(self.tensor_dir, f"{label}.pt")
        all_chunks = torch.load(chunk_path)
        chunk_tensor = all_chunks[chunk_idx]  # shape: (1, 128, 128)

        if self.transform:
            chunk_tensor = self.transform(chunk_tensor)

        label_tensor = torch.zeros(self.num_classes, dtype=torch.float32)
        label_tensor[self.label_to_idx[label]] = 1.0

        return chunk_tensor, label_tensor

In [ ]:
from torch.utils.data import DataLoader

import os

labels = sorted([f.replace(".pt", "") for f in os.listdir(tensor_dir)])
label_to_index = {label: idx for idx, label in enumerate(labels)}

tensor_dir = "Data_BIRDCLEF/birdclef-2025/processed_train_audio"

train_dataset = TensorChunkDataset(
    tensor_dir=tensor_dir,
    label_to_idx=label_to_index,
    transform=None  
)

training_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
import torch
import torch.nn as nn

class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv2d = nn.Conv2d(1,16,kernel_size=3,padding=1)
        self.conv = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=3,padding=1), #1 nur wenn es wirklich grayscale images sind
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16,32,kernel_size=3,padding=1), # 16 weil wir nochmals die input_size auf 32 erhöhen 
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128,num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x 